In [1]:
import os

In [5]:
%pwd

'c:\\Users\\1.1\\Desktop\\Projects\\ML_projects\\Text-Summarizer-NLP-'

In [4]:
os.chdir("../../")

In [6]:
!pip install transformers dataset

  Using cached importlib_resources-6.4.5-py3-none-any.whl.metadata (4.0 kB)
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 1.6/1.6 MB 12.2 MB/s eta 0:00:00


In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [8]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config

In [14]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [15]:
from transformers import AutoTokenizer
from datasets import load_from_disk

class DataTransformation:
    def __init__(self, config):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, example_batch):
        # Tokenize the input dialogue (source text)
        input_encodings = self.tokenizer(example_batch['dialogue'], max_length=1024, truncation=True, padding='max_length')

        # Tokenize the target summary (summary text)
        target_encodings = self.tokenizer(example_batch['summary'], max_length=128, truncation=True, padding='max_length')

        # Return the encoded features for input and labels
        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }

    def convert(self):
        # Load the dataset
        dataset_samsum = load_from_disk(self.config.data_path)

        # Apply the conversion to the dataset
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched=True)

        # Save the transformed dataset to disk
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))


In [47]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2025-01-11 23:19:56,240: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-11 23:19:56,240: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-11 23:19:56,247: INFO: common: created directory at: artifacts]
[2025-01-11 23:19:56,248: INFO: common: created directory at: artifacts/data_transformation]


ValueError: Converting from Tiktoken failed, if a converter for SentencePiece is available, provide a model path with a SentencePiece tokenizer.model file.Currently available slow->fast convertors: ['AlbertTokenizer', 'BartTokenizer', 'BarthezTokenizer', 'BertTokenizer', 'BigBirdTokenizer', 'BlenderbotTokenizer', 'CamembertTokenizer', 'CLIPTokenizer', 'CodeGenTokenizer', 'ConvBertTokenizer', 'DebertaTokenizer', 'DebertaV2Tokenizer', 'DistilBertTokenizer', 'DPRReaderTokenizer', 'DPRQuestionEncoderTokenizer', 'DPRContextEncoderTokenizer', 'ElectraTokenizer', 'FNetTokenizer', 'FunnelTokenizer', 'GPT2Tokenizer', 'HerbertTokenizer', 'LayoutLMTokenizer', 'LayoutLMv2Tokenizer', 'LayoutLMv3Tokenizer', 'LayoutXLMTokenizer', 'LongformerTokenizer', 'LEDTokenizer', 'LxmertTokenizer', 'MarkupLMTokenizer', 'MBartTokenizer', 'MBart50Tokenizer', 'MPNetTokenizer', 'MobileBertTokenizer', 'MvpTokenizer', 'NllbTokenizer', 'OpenAIGPTTokenizer', 'PegasusTokenizer', 'Qwen2Tokenizer', 'RealmTokenizer', 'Refor

In [26]:
# Step 1: Import necessary modules
import os
from pathlib import Path
from dataclasses import dataclass
from transformers import AutoTokenizer
from datasets import load_from_disk
from tqdm import tqdm


# Step 2: Set up configuration and helper methods

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

# Dummy function to simulate the `create_directories` function
def create_directories(directories):
    for directory in directories:
        if not os.path.exists(directory):
            os.makedirs(directory)

# Dummy function to simulate the `read_yaml` function
def read_yaml(filepath):
    # Replace with the actual reading logic or mock for testing
    return {
        'data_transformation': {
            'root_dir': 'artifacts/artifacts/data_ingestion',
            'data_path': 'samsum_dataset',  # This is directly passed in the code below as it was from your previous setup
            'tokenizer_name': 'distilbert-base-uncased'  # Keeping the tokenizer name consistent
        },
        'artifacts_root': 'artifacts'
    }

# Step 3: ConfigurationManager class (same structure)
class ConfigurationManager:
    def __init__(self, config_filepath='config.yaml', params_filepath='params.yaml'):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config['artifacts_root']])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config['data_transformation']

        create_directories([config['root_dir']])

        return DataTransformationConfig(
            root_dir=Path(config['root_dir']),
            data_path=Path(config['data_path']),  # No path to hardcode here as it's handled by your config
            tokenizer_name=Path(config['tokenizer_name'])
        )


# Step 4: DataTransformation class to handle the conversion
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'], max_length=1024, truncation=True)
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length=128, truncation=True)

        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)  # This points to 'dataset_samsum' from your setup
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched=True)
        
        # Save the transformed dataset to disk
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))


# Step 5: Execution
try:
    # Step 5.1: Configuration Manager setup
    config = ConfigurationManager(config_filepath='config.yaml', params_filepath='params.yaml')
    data_transformation_config = config.get_data_transformation_config()

    # Step 5.2: DataTransformation setup and conversion
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    print(f"An error occurred: {e}")


An error occurred: Directory samsum_dataset not found
